In [ ]:
from control.control import *
from forecast_engine.dataset import import_data, missing_values
from forecast_engine.forecast import (
    ols_sm, calculate_metrics, 
    vif, 
    bounds, 
    validate, 
    forward_forecast, 
    rolling_forecast, 
    bayes_model, 
    bayes_forward_forecast
)

from forecast_engine.plotting import (
    actual_vs_fitted, 
    residual_histogram, 
    heteroscedasticity, 
    actual_vs_fitted_bounds, 
    validation_plot_lines, 
    actual_vs_fitted_forecast_bounds, 
    plot_input_vars,
    rolling_forecast_chart,
    sigma_histogram,
    coef_histogram,
    intercept_histogram,
    bayes_forecast_distribution
)

# Import dataset
df = import_data(path="c:/Users/erick/Documents/Python/cpi_fah_fcst/data/cpi_fah_eda_dataset.csv", index_var="date", date_format="%Y-%m-%d")

# Filter by Training to assess missing values
df_training = df.loc[start_date_training : end_date_training].copy()

# Filter columns
df_training = df_training[[dep] + ind + other_vars]

missing_counts = missing_values(df_training, ind)

if missing_counts.empty:
    print("✅ No missing values remain.")
else:
    print("⚠️ Still missing values:")
    print(missing_counts)
    
# Run the training model and append the result to df
df_training, y_fitted, residuals, summary, model, X = ols_sm(df=df_training, ind=ind, dep=dep)

# Run the performance metrics
metrics = calculate_metrics(actual=df_training[dep], predicted=df_training['y_fitted'], round_digits=2)
print(f"MAE:  {metrics['MAE']}")
print(f"MAPE: {metrics['MAPE (%)']}%")

# print training results
actual_vs_fitted(df=df_training, dep='cpi_fah_yoy', fitted='y_fitted')
residual_histogram(df=df_training)
heteroscedasticity(df=df_training)
vif_df = vif(X)
print(vif_df)

# Calculate upper and lower bounds
df_training = bounds(df_training, model, dep, X)

# Chart of actual and fitted with upper and lower bounds
actual_vs_fitted_bounds(df=df_training, dep='cpi_fah_yoy', fitted='y_fitted', end_date=end_date_training)

# Validate the errors with a rolling 6 month forecast vs actuals calculation and chart
validation_df = validate(df=df_training, ind=ind, dep=dep)
validation_plot_lines(df=validation_df)

# forward forecast
forward_values = import_data(path="c:/Users/erick/Documents/Python/cpi_fah_fcst/data/forward_values.csv", index_var="date", date_format=None)
df_full = forward_forecast(forward_values, dep, ind, model, df_training)
actual_vs_fitted_forecast_bounds(df_full)

# plot input vars
plot_input_vars(df=df_full)

# Build forecast chart
rolling_forecast_df = rolling_forecast(df_full)
rolling_forecast_chart(rolling_forecast_df)

# this saves the data in the chart above
rolling_forecast_df.to_csv(f'c:/Users/erick/Documents/Python/cpi_fah_fcst/data/cpi_fah_forecast_month_{forecast_month}.csv', index=True, index_label="month") # create a backup that is not overwritten
rolling_forecast_df.to_csv('c:/Users/erick/Documents/Python/cpi_fah_fcst/data/cpi_fah_forecast_month.csv', index=True, index_label="month") # the main file to build and maintain

# Run Bayes Model
trace = bayes_model(df_training, dep, ind)
sigma_samples = trace.posterior["sigma"].stack(draws=("chain", "draw")).values.flatten()
bayes_data = np.load("posterior_samples.npz")
print(bayes_data.files)
sigma_histogram(sigma_samples)
coef_histogram(coefs_samples=bayes_data['coefs'])
intercept_histogram(intercept_samples=bayes_data['intercept'])

# Bayes forward forecast
posterior_preds = bayes_forward_forecast(forward_values, ind, bayes_data['intercept'], bayes_data['coefs'])

# Bayes forecast distribution 
posterior_preds_array = np.vstack(posterior_preds)

summary_yoy = bayes_forecast_distribution(posterior_preds_array)

# print out results
print(f"Mean: {summary_yoy['mean']:.2f}")
print(f"5th Percentile: {summary_yoy['p5']:.2f}")
print(f"95th Percentile: {summary_yoy['p95']:.2f}")

# Save the output
df_full.index.name='date'
df_full.to_pickle("c:/Users/erick/Documents/Python/cpi_fah_fcst/data/cpi_fah_dataset.pkl")
df_full.to_csv("c:/Users/erick/Documents/Python/cpi_fah_fcst/data/cpi_fah_dataset.csv")
